In [1]:
# Making the necessary imports

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
import cv2 as cv
import keras
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D,MaxPooling2D, Activation, Dropout, BatchNormalization, Input
from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import glob

Using TensorFlow backend.


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount("/content/drive")

gpg: keybox '/tmp/tmpyiqnwtfi/pubring.gpg' created
gpg: /tmp/tmpyiqnwtfi/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
E: Unable to locate package google-drive-ocamlfuse
Mounted at /content/drive


In [3]:
# Importing training images for preprocessing

total_images = 0
major = os.listdir("drive/My Drive/training")
full_path = []
image_labels = []
all_labels = ['donuts', 'salad', 'lasagna', 'pancakes', 'pizza', 'risotto', 'cat', 'dog', 'monkey', 'squirrel', 'daisy', 'dandelion', 'sunflower', 'rose', 'tulip']

print("Loading file structure...\n")
for a in major:
    for b in os.listdir("drive/My Drive/training/" + a):
        full_path.append("drive/My Drive/training/"+a+"/"+b+"/")
        #all_labels.append(b)
        
print("Loading training images...\n")
train_images = []
for i in full_path:
    images_in_folder = 0
    label = i.split('/')[4]
    for file in glob.glob(i+"*.jpg"):
        img = cv.cvtColor(cv.imread(file),cv.COLOR_BGR2RGB)
        img = cv.resize(img, (192,192))
        total_images+=1
        train_images.append(img)
        image_labels.append(all_labels.index(label))
        images_in_folder += 1
    print("The total number of images in %s = %d" % (i,images_in_folder))
print("The total number of images in data = " + str(total_images))

Loading file structure...

Loading training images...

The total number of images in drive/My Drive/training/animals/cat/ = 289
The total number of images in drive/My Drive/training/animals/squirrel/ = 386
The total number of images in drive/My Drive/training/animals/dog/ = 297
The total number of images in drive/My Drive/training/animals/monkey/ = 299
The total number of images in drive/My Drive/training/flowers/sunflower/ = 552
The total number of images in drive/My Drive/training/flowers/tulip/ = 744
The total number of images in drive/My Drive/training/flowers/dandelion/ = 789
The total number of images in drive/My Drive/training/flowers/daisy/ = 577
The total number of images in drive/My Drive/training/flowers/rose/ = 588
The total number of images in drive/My Drive/training/food/risotto/ = 364
The total number of images in drive/My Drive/training/food/lasagna/ = 348
The total number of images in drive/My Drive/training/food/pizza/ = 362
The total number of images in drive/My Driv

In [0]:
# Reducing pixel values range to 0 to 1

X_train = np.array(train_images, np.float32) / 255.

In [5]:
# One hot encoding for labels

image_labels = to_categorical(image_labels)
print (image_labels.shape)

(6406, 15)


In [0]:
# Normalization

mean_img = X_train.mean(axis=0)
std_dev = X_train.std(axis = 0)
X_norm = (X_train - mean_img)/ std_dev
X_norm, image_labels = shuffle(X_norm, image_labels, random_state=0)

In [7]:
# Creating train validation split

Xtrain, Xvalid, Ytrain, Yvalid = train_test_split(X_norm, image_labels, test_size=0.2, random_state=7)
print(Xtrain.shape)


(5124, 192, 192, 3)


In [0]:
model = Sequential()
model.add(BatchNormalization(input_shape=Xtrain.shape[1:]))
model.add(Conv2D(32, kernel_size=(3, 3), activation= 'relu',padding= 'same'))
model.add(Conv2D(32, kernel_size=(3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size=(3, 3), activation= 'relu', padding='same'))
model.add(Conv2D(64, kernel_size=(3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(3, 3), activation= 'relu', padding='same'))
model.add(Conv2D(128, kernel_size=(3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(15, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')

In [11]:
trained_model = model.fit(Xtrain, Ytrain, epochs = 20, shuffle = True, batch_size = 100)

Epoch 1/20
5124/5124 [==============================] - 24s 5ms/step - loss: 2.3453 - acc: 0.1786
Epoch 2/20
5124/5124 [==============================] - 24s 5ms/step - loss: 1.9583 - acc: 0.3062
Epoch 3/20
5124/5124 [==============================] - 24s 5ms/step - loss: 1.7116 - acc: 0.3854
Epoch 4/20
5124/5124 [==============================] - 24s 5ms/step - loss: 1.5716 - acc: 0.4379
Epoch 5/20
5124/5124 [==============================] - 24s 5ms/step - loss: 1.4533 - acc: 0.4781
Epoch 6/20
5124/5124 [==============================] - 23s 5ms/step - loss: 1.3832 - acc: 0.5100
Epoch 7/20
5124/5124 [==============================] - 23s 5ms/step - loss: 1.2868 - acc: 0.5480
Epoch 8/20
5124/5124 [==============================] - 23s 5ms/step - loss: 1.2389 - acc: 0.5669
Epoch 9/20
5124/5124 [==============================] - 23s 5ms/step - loss: 1.1561 - acc: 0.5943
Epoch 10/20
5124/5124 [==============================] - 23s 5ms/step - loss: 1.1204 - acc: 0.6054
Epoch 11/20
5124/51

In [0]:
model.save('model.h5')

In [25]:
!ls
from google.colab import files
#files.download('model.h5')
!mv model.h5  drive/My\ Drive

adc.json  drive  model.h5  sample_data
